In [ ]:
def convert_Date(italian_date):
    # Mapeo de nombres de meses en italiano a números de mes
    meses_italiano_a_numero = {
        'gennaio': '01',
        'febbraio': '02',
        'marzo': '03',
        'aprile': '04',
        'maggio': '05',
        'giugno': '06',
        'luglio': '07',
        'agosto': '08',
        'settembre': '09',
        'ottobre': '10',
        'novembre': '11',
        'dicembre': '12'
    }

    # Dividir la fecha italiana en partes
    partes_fecha = italian_date.split()

    # Obtener día, mes y año
    dia = partes_fecha[0]
    mes = meses_italiano_a_numero[partes_fecha[1]]
    año = partes_fecha[2]

    # Formatear la fecha en el formato deseado
    fecha_formateada = f"{año}-{mes}-{dia}"

    return fecha_formateada

In [48]:
import csv
import os

def create_csv(keyword, from_date, to_date, the_urls):

  keyword = keyword.replace(" ", "_")

  csv_file_path = '_'.join([keyword, from_date, to_date]) + '.csv'

# Transform each URL into a dictionary
  data = [{"URL": url} for url in the_urls]

  if os.path.exists(csv_file_path):
    # Si existe, abre el archivo en modo de escritura
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())

        # Escribe los nuevos datos en el archivo
        for row in data:
            writer.writerow(row)

    print(f"Data added to existing file: {csv_file_path}")
  else:
# If it does not exist, create a new file and write the data
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())

        # Write the headers
        writer.writeheader()

        # Write the data
        for row in data:
            writer.writerow(row)

    print(f"Nuevo archivo CSV creado: {csv_file_path}")

In [85]:
import requests
from bs4 import BeautifulSoup
from lxml import html


def get_the_total_number_of_pages(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.text,'html.parser')
  root = html.fromstring(str(soup))
  try:
    paragraph_text = root.xpath('//*[@id="lista-risultati"]/div/p/text()[2]')[0]
    total_pages = paragraph_text.split()[1]
  except:
    total_pages = 0

  return int(total_pages)

def get_date_of_next_period(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.text,'html.parser')
  root = html.fromstring(str(soup))

  dates = root.xpath('//*[@id="lista-risultati"]/article/aside/a')[-1]
  date = dates.split()[2]
  return dates


In [56]:

def urls_generator(keyword, from_date, to_date, modality, number_of_pages):

    keyword = keyword.replace(" ", "+")

    url_base = "https://ricerca.repubblica.it/ricerca/repubblica?query={}&fromdate={}&todate={}&sortby=adate&author=&mode={}&page={}"

    urls = []

    # Generate URLs
    for page in range(1, number_of_pages + 1):
        url = url_base.format(keyword, from_date, to_date, modality, page)
        print(url)
        urls.append(url)
    return urls

In [55]:
import csv
import os

def scraper(keyword, from_date, to_date, modality):

    # returns an array
    urls = urls_generator(keyword, from_date, to_date, modality,1)

    # returns an number
    total_number_of_pages = get_the_total_number_of_pages(urls[0]) # Example 140

    # first possibility
    if total_number_of_pages == 0:
        return print('there is no news with the entry entered')

    # second possibility
    elif  total_number_of_pages <= 50:
        the_urls = urls_generator(keyword, from_date, to_date, modality,total_number_of_pages)

        create_csv(keyword, from_date, to_date, the_urls)

    # third possibility
    else:
        max_pages_per_keyword = 50
        the_urls = urls_generator(keyword, from_date, to_date, modality, max_pages_per_keyword)

        create_csv(keyword, from_date, to_date, the_urls)


scraper("tasa", "1984-01-01", "2019-01-01", "all")

https://ricerca.repubblica.it/ricerca/repubblica?query=tasa&fromdate=1984-01-01&todate=2019-01-01&sortby=adate&author=&mode=all&page=1
https://ricerca.repubblica.it/ricerca/repubblica?query=tasa&fromdate=1984-01-01&todate=2019-01-01&sortby=adate&author=&mode=all&page=1
https://ricerca.repubblica.it/ricerca/repubblica?query=tasa&fromdate=1984-01-01&todate=2019-01-01&sortby=adate&author=&mode=all&page=2
https://ricerca.repubblica.it/ricerca/repubblica?query=tasa&fromdate=1984-01-01&todate=2019-01-01&sortby=adate&author=&mode=all&page=3
Data added to existing file: tasa_1984-01-01_2019-01-01.csv


In [86]:
get_date_of_next_period('https://ricerca.repubblica.it/ricerca/repubblica?query=asa&fromdate=1984-01-01&todate=1986-01-01&sortby=adate&author=&mode=all&page=1')

24 aprile 1984
30 agosto 1984
13 dicembre 1984
25 giugno 1985
28 dicembre 1985
